# Calibration Procedure

* Compute center offset:
 - Set $\lambda_{\rm center}$ to set of known spectral lines
 - Measure pixel position of each: 
 - pick one and determine offset_adjustment each to determine central pixel $n_o$
 
|   $\lambda_{\rm center}$ | Pixel |           
| ----------------------:  |:------:|
| 0   nm                   | 5.2 | 
| 445 nm                   | 6.22      |  
| 901 nm                   | 3.1      | 


 
* Compute spectrometer dispersion calibration. i.e: angles/length ($\ f_L, \delta, \gamma$)
 * Move known spectral line $\lambda_o$ to left and right sides of detector
 * record $\lambda_{\rm center}$ and pixel position for each 
 * Compute best fit of $\ f_{\rm calib}$


| $\lambda_o$   | Side | $\lambda_{\rm center}$| Pixel  |
| ------------- | ---- |:----------------------|-------:|
| 809.4 nm      | R    |729.4910 nm            |508     |
| 809.4 nm      | L    |899.5830 nm            |  4     |
| ...           | ...  | ...                   |...     |



# Optimization Function

Optimize for 3 parameters:
 * $f_L$: Focal length of spectrometer
 * $\delta$: Detector angle (The angle of the image plane relative to the plane perpendicular to the spectrograph focal axis at the center of the image plane)
 * $\gamma$: inclusion angle

from experiment:
 * $n =  n_{px} - n_o$: Pixel from central pixel
 * $\lambda_{\rm center}$: Wavelength of center pixel 
 * $\lambda_p$: Wavelength of pixel $n$
 
Fixed Constants:
 * $m$: Diffraction order (typically one)
 * $x_{\rm pixel}$: pixel size
 * $d_{grating}$: Grating pitch (1/(groves / mm))
    
residual: (wl,  wl_p, n, f, delta,gamma)

We measure pixel position ($n$) of a known wavelength ($\lambda_p$) for multple peaks and spectrometer positions and find the best fit parameters $\ f_L, \delta, \gamma$:

$$ \lambda_p = f_{\rm calib} ( n,  \lambda_{\rm center}, 
    \underbrace{m, x_{\rm pixel}, d_{\rm grating}}_{\rm spec\ params}, 
    \overbrace{f_L,\ \ \delta,\ \ \gamma}^{\rm Calibration\ params} ) $$

$$ \lambda_p = \frac{d}{m} \cdot \left[ \sin( \psi - \frac{\gamma}{2}) + \sin(\psi+\frac{\gamma}{2} + \eta) \right]$$

Where

$$ \psi = \arcsin \left[ \frac{ m\ \lambda_{\rm center} } { 2\ d_{\rm grating} \cos(\frac{\gamma}{2})} \right] $$

$$ \eta = \arctan \left[ \frac{ n\ x_{pixel} \cos{\delta}} {f_L + n\ x_{pixel} \sin(\delta)} \right]$$

$$n =  n_{px} - n_o$$


Note: $nx_{\rm pixel}$ is the distance from the center


Hg has strong persistent line at 
194.223,
435.8328,
912.288,
1529.582nm. The last two are perticularly useful for NIR
https://physics.nist.gov/PhysRefData/Handbook/Tables/mercurytable2.htm

In [1]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint
import glob as glob

import h5py
def loadH5_dset(file_name, dset_name):
    h5_file = h5py.File(file_name)
    dset_ =  h5_file.visititems(lambda name,node:dset_visitfunction(name, node,dset_name=dset_name))
    h5_file.close()
    return dset_
def dset_visitfunction(name,node,dset_name):
    if isinstance(node, h5py.Dataset):
        if dset_name in name:
            return np.array(node)
def loadH5_attr(file_name, attr_name):
    h5_file = h5py.File(file_name)
    dset_ =  h5_file.visititems(lambda name,node:attr_visitfunction(name, node,attr_name=attr_name))
    h5_file.close()
    return dset_
def attr_visitfunction(name,node,attr_name):
    for key, val in node.attrs.items():
        if key == attr_name:
            return val   
        
        
def wl_p_calib(px, n0, offset_adjust, wl_center, m_order, d_grating, x_pixel, f, delta, gamma):
    #consts
    #d_grating = 1./150. #mm
    #x_pixel   = 16e-3 # mm
    #m_order   = 1 # diffraction order, unitless
    n = px - (n0+offset_adjust*wl_center)

    psi = np.arcsin( m_order* wl_center / (2*d_grating*np.cos(gamma/2)))
    eta = np.arctan(n*x_pixel*np.cos(delta) / (f+n*x_pixel*np.sin(delta)))

    return ((d_grating/m_order)
                    *(np.sin(psi-0.5*gamma)
                      + np.sin(psi+0.5*gamma+eta)))

# Camera

In [2]:
x_pixel = 25e3 # in nm

## Grating 1
### from center wl, go +/-60 nm for left/right measurement.

In [51]:
d_grating = (1.0/600)*1e6
m_order   = 1

In [5]:
#offset data
wl_center = np.array([  0,763.5030,912.2880])
px =        np.array([493,     490,     491])
n0 = 517
print('n0=',n0)
#lines
n_px      = np.array([    3,     1013])
wl_actual = np.array([912.288,912.288])
wl_center = np.array([980   ,     850])

n0= 517


## Grating 2

In [6]:
d_grating = (1.0/600)*1e6
m_order = 1
#offset data
wl_center = np.array([  0,763.5030,912.2880])
px =        np.array([489,     487,     488])
n0 = px.mean()
print('n0=',n0)
#lines
n_px      = np.array([    195,    591, 619     ])
wl_actual = np.array([912.288,912.288, 912.288 ])
wl_center = np.array([1000   ,    900, 950     ])


n0= 488.0


## Grating 3 (calibrated around 912.3: 790 -1080)

In [3]:
d_grating = (1.0/300)*1e6
m_order = 1

### offset

In [25]:
n0 = 493
def spectral_median(spec,wls, count_min=200):
    int_spec = np.cumsum(spec)
    total_sum = int_spec[-1]
    if total_sum > count_min:
        pos = int_spec.searchsorted( 0.5*total_sum)
        wl = wls[pos]
    else:
        wl = np.NaN
    return wl

fnames = glob.glob('data/grating_3/n0_adjustment/*winspec*')
wl_center = np.zeros(len(fnames))
n_px = wl_center.copy()
wl_actual = wl_center.copy()
for ii,fname in enumerate(fnames):
    h5_file = h5py.File(fname)
    s = np.s_[470:530]
    spec = loadH5_dset(fname,'spectrum')[0,0,s]
    px = spectral_median(spec, np.arange(1024)[s])
    print(loadH5_attr(fname, 'center_wl'), px)

912.297 493
1529.578 491


In [26]:
offset_center_px = [493.,  491.]
offset_center_wl = [912.297, 1529.578]
offset_adjust = (np.diff(offset_center_px)/np.diff(offset_center_wl)).mean()
print(offset_adjust)

-0.00324001548727


### dispersion calibration

In [27]:
# here n_px is computed automatically using argmax() which is not always a valid approach 
fnames = glob.glob('data/grating_3/*winspec*')
wl_center = np.zeros(len(fnames))
n_px = wl_center.copy()
wl_actual = wl_center.copy()
for ii,fname in enumerate(fnames):
    h5_file = h5py.File(fname)
    spec = loadH5_dset(fname,'spectrum')
    n_px[ii] = spec.argmax()
    wl_center[ii] = loadH5_attr(fname, 'center_wl')
    wl_actual[ii] = 912.3
    
print(n_px)
print(wl_center)
print(wl_actual)

[ 540.  890.  149.]
[ 899.992  809.993  999.994]
[ 912.3  912.3  912.3]


In [31]:
from scipy.optimize import least_squares

def fit_residual(
                # optimization parameters
                params, 
                # other params and data
                px, n0, offset_adjust, wl_center, m_order, d_grating, x_pixel,
                wl_actual
                ):
    (f, delta, gamma,) = params
    wl_model = wl_p_calib(px, n0, offset_adjust, wl_center, m_order, d_grating, x_pixel, f, delta, gamma)
    return wl_model - wl_actual
    
initial_guess = (300e6, 0.0, 0.0)

s=np.s_[:]
#s = [0,-2]
kwargs = dict(
    px=n_px[s], 
    n0=n0,
    offset_adjust = offset_adjust,
    wl_center=wl_center[s],
    m_order=1,
    d_grating=d_grating,
    x_pixel=x_pixel,
    wl_actual = wl_actual[s]
)

result = least_squares(fit_residual, initial_guess, kwargs=kwargs, xtol=1e-15, gtol=1e-15, ftol=1e-15)


In [32]:
calibration = [result.x[0],
               result.x[1],
               result.x[2],
               n0,
               offset_adjust,
               d_grating, 
               x_pixel]
print('f, delta, gamma, n0, offset_adjust, d_grating, x_pixel')
print(calibration)

f, delta, gamma, n0, offset_adjust, d_grating, x_pixel
[321689128.87573242, 0.088326333056814615, -0.072049816199315248, 493, -0.0032400154872740295, 3333.3333333333335, 25000.0]


In [33]:
#Grating 3 without adjustment
#[299999998.98569852, -0.18306223592186061, 0.42417029798171096, 493.0, 3333.3333333333335, 25000.0]
[299999999.04755104, -0.36295464599714206, 0.042058985744931922, 498, -0.0032400154872740295, 3333.3333333333335, 25000.0]

[299999999.04755104,
 -0.36295464599714206,
 0.04205898574493192,
 498,
 -0.0032400154872740295,
 3333.3333333333335,
 25000.0]